In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [2]:
y2015 = pd.read_csv("LoanStats3d.csv", skipinitialspace=True, header=1)
y2015.head()

C:\Users\andre\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [3]:
categorical = y2015.select_dtypes(include=["object"])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
421097
term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


In [4]:
# Convert ID and Interest Rate to numeric
y2015["id"] = pd.to_numeric(y2015["id"], errors="coerce")
y2015["int_rate"] = pd.to_numeric(y2015["int_rate"].str.strip("%"), errors="coerce")

# Drop other columns with many unique variables
y2015.drop(["url", "emp_title", "zip_code", "earliest_cr_line", "revol_util", "sub_grade",
           "addr_state", "desc"], 1, inplace=True)

In [5]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
y2015 = y2015[:-2]
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421090,36371250.0,39102635.0,10000.0,10000.0,10000.0,36 months,11.99,332.10,B,8 years,...,0.0,1.0,100.0,100.0,0.0,0.0,32950.0,25274.0,9200.0,15850.0
421091,36441262.0,39152692.0,24000.0,24000.0,24000.0,36 months,11.99,797.03,B,10+ years,...,0.0,2.0,56.5,100.0,0.0,0.0,152650.0,8621.0,9000.0,0.0
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0


In [7]:
rfc = ensemble.RandomForestClassifier()
X = y2015.drop("loan_status", 1)
y = y2015["loan_status"]
X = pd.get_dummies(X)
X = X.dropna(axis=1)

start_time = time.time()
print(cross_val_score(rfc, X, y, cv=10))
end_time = time.time()
print("\n--- %s seconds ---" % (end_time - start_time))

[0.98021895 0.98064639 0.98162    0.98162    0.97347423 0.97504156
 0.96266831 0.98057422 0.97969459 0.97971785]

--- 205.34264993667603 seconds ---


In [8]:
from sklearn.preprocessing import StandardScaler

X = y2015.drop("loan_status", 1)
y = y2015["loan_status"]
X = pd.get_dummies(X)
X = X.dropna(axis=1)

scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)

## PCA: 2 Components

In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(scaled_X)
x_pca = pca.transform(scaled_X)

In [10]:
scaled_X.shape

(421095, 201)

In [11]:
x_pca.shape

(421095, 2)

In [13]:
rfc = ensemble.RandomForestClassifier()
X = x_pca
y = y2015["loan_status"]

start_time = time.time()
print(cross_val_score(rfc, X, y, cv=10))
end_time = time.time()
print("\n--- %s seconds ---" % (end_time - start_time))

[0.64928404 0.6469806  0.64557954 0.64503336 0.63956305 0.64381382
 0.63428246 0.6270156  0.62528796 0.62067164]

--- 140.13916730880737 seconds ---


## PCA: 5 Components

In [14]:
pca = PCA(n_components=5)
pca.fit(scaled_X)
x_pca = pca.transform(scaled_X)
x_pca.shape

(421095, 5)

In [15]:
rfc = ensemble.RandomForestClassifier()
X = x_pca
y = y2015["loan_status"]

start_time = time.time()
print(cross_val_score(rfc, X, y, cv=10))
end_time = time.time()
print("\n--- %s seconds ---" % (end_time - start_time))

[0.86549833 0.87677804 0.88962504 0.8950393  0.90078366 0.90339587
 0.9036548  0.89543803 0.87629135 0.81567948]

--- 162.26869535446167 seconds ---


##  PCA: 10 Components

In [16]:
pca = PCA(n_components=10)
pca.fit(scaled_X)
x_pca = pca.transform(scaled_X)
x_pca.shape

(421095, 10)

In [17]:
rfc = ensemble.RandomForestClassifier()
X = x_pca
y = y2015["loan_status"]

start_time = time.time()
print(cross_val_score(rfc, X, y, cv=10))
end_time = time.time()
print("\n--- %s seconds ---" % (end_time - start_time))

[0.91866733 0.9317518  0.93462516 0.93702358 0.93526478 0.93417241
 0.93590444 0.93189104 0.93673261 0.93138745]

--- 247.85121965408325 seconds ---


This actually looks pretty good.